In [8]:
import pandas as pd
import os
import shutil
import random
import subprocess
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification,BertAdam
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")
random.seed(42)

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
src = pd.read_csv("~/mt-dnn/toxic_data/train.csv")

In [4]:
train, test = train_test_split(src,test_size=0.05)

In [17]:
pd.options.mode.chained_assignment = None  # default='warn'

In [18]:
def full_tokenizer(raw_string):
    raw_string=raw_string[0]
    tokens = tokenizer.tokenize(raw_string)
    if len(tokens)>510:
        tokens = tokens[0:510]
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    return tokenizer.convert_tokens_to_ids(tokens)

In [26]:
def toxic_prep(sample,path_to_prepared=''):
    how_many_datapoints = sample.shape[0]
    sample = sample[['id',"target",'comment_text']]        
    sample['token_id'] = sample[['comment_text']].apply(full_tokenizer,axis = 1)
    sample['type_id'] = sample[['token_id']].apply(lambda x:[0]*len(x[0]),axis = 1)
    sample.drop(['comment_text'], axis=1, inplace=True)
    sample = sample.rename(columns={'id': 'uid', 'target': 'label'})
    sample.to_json(path_to_prepared, orient='records', lines=True)
toxic_prep(test,"/home/jupyter/mt-dnn/data/mt_dnn/toxic_test.json")
toxic_prep(train,"/home/jupyter/mt-dnn/data/mt_dnn/toxic_train.json")

In [4]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [5]:
file_len("/home/jupyter/mt-dnn/data/mt_dnn/toxic_test.json")

90244

In [7]:
file_len("/home/jupyter/mt-dnn/data/mt_dnn/toxic_train.json")

1714630

In [14]:
def cut_file(file,max_length):
    with open(file) as f, open("/home/jupyter/mt-dnn/tmp.txt", "w") as out:
        out.writelines(f.readlines()[:max_length])
    os.remove(file)
    shutil.move("/home/jupyter/mt-dnn/tmp.txt",file)    

In [15]:
# trying a shorter train set, for more reasonable training time
cut_file('/home/jupyter/mt-dnn/data/mt_dnn/toxic_train.json',214328)

In [ ]:
#Not working... Cuda ran out of memory after 3 hours...
#I'll try instead to divide the train set into much smaller batches, to have shorter epochs

In [31]:
for i in range(3):
    print(i)

0
1
2


In [36]:
def smaller_train_sets(file,file_size=1714630,factor=30):
    with open(file) as f:
        lines=f.readlines()
        mini_train_size = int(file_size/factor)
        for i in range(factor):
            with open("/home/jupyter/mt-dnn/data/mt_dnn/toxic_train" + str(i) + ".json", "w") as out:
                out.writelines(lines[i*mini_train_size:(i+1)*mini_train_size])

In [37]:
smaller_train_sets("/home/jupyter/mt-dnn/data/mt_dnn/toxic_train_full.json")

In [30]:
[0]*6

[0, 0, 0, 0, 0, 0]